# 2018-06-08 - Comparaison modèle/humain

# Le convo

In [1]:
import torch
import torchvision
from torchvision import transforms, datasets

data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

#train
train_set = datasets.ImageFolder(root='16_clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=4, shuffle=True,
                                             num_workers=1, drop_last = True)

#test
test_set = datasets.ImageFolder(root='16_clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=4,shuffle=True,
                                             num_workers=1, drop_last = True)

In [2]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 20)
        self.pool = nn.MaxPool2d(2,2)

        self.fc3 = nn.Linear(17496,1000)
        self.dropout = nn.Dropout(0.2)

        self.outlayer = nn.Linear(1000,16)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))

        x = x.view(x.size(0), -1) #reshape from conv to linear

        x = F.leaky_relu(self.fc3(x))
        x = self.dropout(x)

        x = self.outlayer(x)
        return x
        
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(20, 20), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (fc3): Linear(in_features=17496, out_features=1000, bias=True)
  (dropout): Dropout(p=0.2)
  (outlayer): Linear(in_features=1000, out_features=16, bias=True)
)


In [3]:
#freeze layers after and including freezing_layer+1 (layers start at 0)
def freeze_layers(freezing_layer, nn_model = model) :
    for count,child in enumerate(model.children()) :
        if count < freezing_layer+1 : #to freeze at iteration 1
            print("Layer no. %s -- %s -- NOT FROZEN"% (count,child))
            for param in child.parameters():
                param.requires_grad = True
        else :
            print("Layer no. %s -- %s -- FROZEN"%(count, child))
            for param in child.parameters():
                param.requires_grad = False

#freeze all the layers except the unfrozen one
def freeze_all_layers(unfrozen_layer, nn_model = model) :
    for count,child in enumerate(model.children()) :
        if count == unfrozen_layer :
            print("Layer no. %s -- %s -- NOT FROZEN"% (count,child))
            for param in child.parameters():
                param.requires_grad = True
        else :
            print("Layer no. %s -- %s -- FROZEN"%(count, child))
            for param in child.parameters():
                param.requires_grad = False

#unfreeze every layers
def layers_microwave(nn_model = model) :
    for count,child in enumerate(model.children()) :
        for param in child.parameters():
            param.requires_grad = True

In [4]:
import torch.optim as optim
import time

#unfreeze before starting
layers_microwave(model)

criterion = nn.CrossEntropyLoss() #loss criterion
optimizer = optim.SGD(params = model.parameters(),lr=0.001, momentum=0.9)
epochs = 2 #nbr of epochs per layer
model_size = 4 #nbr of layers

print_interval = 50 #prints every p_i*4
tempo = []
acc = []

start_time = time.time()
print("Started training")

for epoch in range(epochs):  # nbr epochs
    for batch_idx, (data, target) in enumerate(train_loader): #nbr batch,in,out
        data, target = Variable(data), Variable(target)
        #On resize pour la sortie


        #init l'entrainement
        optimizer.zero_grad()
        net_out = model(data)

        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()

        #afficher la progression
        if batch_idx % print_interval == 0:
            #le print statement le plus illisible du monde
            print('Epoch: {} [{}/{} ({:.0f}%)]\t\tLoss: {:.6f}'.format(
                    epoch+1, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
    tempo.append(epoch)
    acc.append(loss.data[0])


print("Finished training in  %.3f seconds " % (time.time() - start_time))

Started training
Epoch: 1 [0/3840 (0%)]		Loss: 2.755982
Epoch: 1 [200/3840 (5%)]		Loss: 2.788765
Epoch: 1 [400/3840 (10%)]		Loss: 2.562751
Epoch: 1 [600/3840 (16%)]		Loss: 2.826652
Epoch: 1 [800/3840 (21%)]		Loss: 1.932462
Epoch: 1 [1000/3840 (26%)]		Loss: 2.359261
Epoch: 1 [1200/3840 (31%)]		Loss: 2.189623
Epoch: 1 [1400/3840 (36%)]		Loss: 1.764187
Epoch: 1 [1600/3840 (42%)]		Loss: 1.294152
Epoch: 1 [1800/3840 (47%)]		Loss: 1.219923
Epoch: 1 [2000/3840 (52%)]		Loss: 0.351010
Epoch: 1 [2200/3840 (57%)]		Loss: 1.130805
Epoch: 1 [2400/3840 (62%)]		Loss: 0.171190
Epoch: 1 [2600/3840 (68%)]		Loss: 0.101823
Epoch: 1 [2800/3840 (73%)]		Loss: 0.440843
Epoch: 1 [3000/3840 (78%)]		Loss: 0.125603
Epoch: 1 [3200/3840 (83%)]		Loss: 0.105759
Epoch: 1 [3400/3840 (89%)]		Loss: 1.222404
Epoch: 1 [3600/3840 (94%)]		Loss: 0.343623
Epoch: 1 [3800/3840 (99%)]		Loss: 0.057144
Epoch: 2 [0/3840 (0%)]		Loss: 0.019357
Epoch: 2 [200/3840 (5%)]		Loss: 0.010543
Epoch: 2 [400/3840 (10%)]		Loss: 0.138879
Epoch: 2 [

In [5]:
# delete
test_loss = 0
correct = 0
model = model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)

    net_out = model(data)
    
    #somme des pertes du batch
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1] #prediction
    correct += pred.eq(target.data).sum() #output du réseau

test_loss /= len(test_loader.dataset) #loss = loss/length set
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0304, Accuracy: 539/576 (94%)



## Convo - Psychophysique

In [1]:
exp_info = 'convo_model'
print (exp_info)

convo_model


On défini la génération de MotionClouds :

In [7]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = './tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

On définit les paramètres et on teste :

In [68]:
from psychopy import visual, core, event
import MotionClouds as MC
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
from PIL import Image

test_length = 150
MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
model = model.eval()


ans_list = []
N_B_thetas = 7
B_thetas = np.pi*np.logspace(-6, -1, N_B_thetas, base=2)
    
std_theta = np.pi/6

for trial in range(test_length):
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2

    varimgmc2 = data_transform(Image.open(MC2))
    #varimgmc2 = data_transform(Image.open('16_clouds_easy/0.0/B0 1.40625.png'))
    varimgmc2 = Variable(varimgmc2)
    varimgmc2 = varimgmc2.unsqueeze(0)
    
    net_ans = model(varimgmc2)
    pred = net_ans.data.max(1)[1] #prediction
    #print(pred)
    #correct = (np.sign(theta) > 0) and (pred[0]>8)
    if np.sign(theta) > 0 and pred[0] > 8 :
        correct = True
    if np.sign(theta) < 0 and pred[0] <= 8:
        correct = True
    else :
        correct = False
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ', pred[0], '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi))

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    if pred[0] >8 :
        ans_list.append([trial, theta, 'left', B_theta])
    if pred[0] <= 8 :
        ans_list.append([trial, theta, 'right', B_theta])


At trial  0 Angle= -16.898 answer is  9 (correct= False ); bandwidth= 15.910
At trial  1 Angle= 26.472 answer is  5 (correct= False ); bandwidth= 2.812
At trial  2 Angle= -23.481 answer is  11 (correct= False ); bandwidth= 50.511
At trial  3 Angle= 13.108 answer is  7 (correct= False ); bandwidth= 2.812
At trial  4 Angle= -24.179 answer is  10 (correct= False ); bandwidth= 5.011
At trial  5 Angle= -45.000 answer is  12 (correct= False ); bandwidth= 28.348
At trial  6 Angle= 10.537 answer is  7 (correct= False ); bandwidth= 5.011
At trial  7 Angle= 14.884 answer is  6 (correct= False ); bandwidth= 15.910
At trial  8 Angle= 44.189 answer is  4 (correct= False ); bandwidth= 2.812
At trial  9 Angle= -43.331 answer is  11 (correct= False ); bandwidth= 15.910
At trial  10 Angle= 8.865 answer is  8 (correct= False ); bandwidth= 28.348
At trial  11 Angle= 24.869 answer is  5 (correct= False ); bandwidth= 28.348
At trial  12 Angle= 24.050 answer is  5 (correct= False ); bandwidth= 50.511
At tri

At trial  107 Angle= -10.301 answer is  8 (correct= True ); bandwidth= 5.011
At trial  108 Angle= 45.000 answer is  4 (correct= False ); bandwidth= 2.812
At trial  109 Angle= -3.178 answer is  8 (correct= True ); bandwidth= 15.910
At trial  110 Angle= -45.000 answer is  11 (correct= False ); bandwidth= 2.812
At trial  111 Angle= 21.120 answer is  6 (correct= False ); bandwidth= 15.910
At trial  112 Angle= -29.894 answer is  10 (correct= False ); bandwidth= 15.910
At trial  113 Angle= -45.000 answer is  11 (correct= False ); bandwidth= 5.011
At trial  114 Angle= 0.196 answer is  7 (correct= False ); bandwidth= 8.929
At trial  115 Angle= -44.683 answer is  11 (correct= False ); bandwidth= 8.929
At trial  116 Angle= 14.588 answer is  5 (correct= False ); bandwidth= 28.348
At trial  117 Angle= -10.200 answer is  8 (correct= True ); bandwidth= 2.812
At trial  118 Angle= 9.619 answer is  9 (correct= False ); bandwidth= 90.000
At trial  119 Angle= -19.439 answer is  9 (correct= False ); bandw

In [29]:
import os
os.listdir('16_clouds_easy')
def pick_random_cloud() :
    theta = random.randint(5, 9)
    theta_dir = os.listdir('16_clouds_easy')
    theta_chosen = theta_dir[theta]
    
    b_theta_dir = os.listdir('16_clouds_easy/%s' % theta_chosen)
    b_theta = random.randint(0, len(b_theta_dir))
    b_theta_chosen = b_theta_dir[b_theta]
    
    print(theta_chosen, b_theta_chosen)
    value_b_theta = b_theta_chosen.split(' ')
    value_b_theta = value_b_theta[1].split('.png')
    value_b_theta = float(value_b_theta[0])
    print(value_b_theta)
    
    return theta_chosen, b_theta_chosen, value_b_theta
    
pick_random_cloud()

2.722713633111154 B0 4.295219195298584.png
4.295219195298584


('2.722713633111154', 'B0 4.295219195298584.png', 4.295219195298584)

In [ ]:
import pickle
pickle.dump(ans_list, open('../psychophysics_data/Psy_discrim_final_%s.p' % exp_info, 'wb'))

# Le Ring

In [ ]:
"""
Created on Tue May 15 11:25:41 2018

@author: hugo
"""
from torch.autograd import Variable
import torch 
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
import torch.nn.functional as F

#Transform
data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((128,128)),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

#Train
train_set = datasets.ImageFolder(root='16_clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=8, shuffle=True,
                                             num_workers=1)

#Test
test_set = datasets.ImageFolder(root='16_clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=8,shuffle=False,
                                             num_workers=1)

###################################################################################

# Hyper-parameters
sequence_length = 49
input_size = 49
hidden_size = 128
num_layers = 2
num_classes = 16
batch_size = 8
num_epochs = 60
learning_rate = 0.003

###################################################################################

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout = 0.3)
        self.conv1 = nn.Conv2d(1,8,30)
        self.conv2 = nn.Conv2d(1,8,30)
        self.pool1 = nn.MaxPool2d(2,2)
        self.pool2 = nn.MaxPool2d(2,2)
        #self.drop = nn.Dropout(0.2)

        self.fc = nn.Linear(113 , 16)  # 2 for bidirection
    
    def forward(self, x):
        #print('Input %s' % str(x.size()))

        out = self.pool1(F.relu(self.conv1(x)))
        #print('Convoluted %s' % str(x.size()))

        # Init
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        h0 = Variable(h0).cuda()
        c0 = Variable(c0).cuda()
        
        out = out[:,-1,:,:]
        #print('Resized for RNN %s' % str(x.size()))
        
        # LSTM forward
        out, _ = self.rnn(out, (h0,c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        #print('After RNN %s' % str(out.size()))
        
        out = out.unsqueeze(1)
        #print(out.size())
        out = self.pool2(F.relu(self.conv2(out)))
        #print(out.size())
        out = out[:,-1,:,:]
        #Dropout
        #out = self.drop(out)
        #print(out.size())
        
        out = out[:,-1,:]
        # LSTM output
        out = self.fc(out)
        #print('Reshaped for output %s \n'%  str(out.size()))

        return out

model = BiRNN(input_size, hidden_size, num_layers, num_classes).cuda()
print(model)

###################################################################################

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

###################################################################################

#plotting list
loss_list = []
time_list = []
t = 0

print("Start training")
# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.data[0]))
            loss_list.append(loss.data[0])
            time_list.append(t)
            t+=1

correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    labels = Variable(labels).cuda()

    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += predicted.eq(labels.data).sum()

print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(time_list, loss_list)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss for LSTM-CNN')
plt.show()
            

In [ ]:
#NOW FOR THE PSYCHOPHYSICS
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

exp_info = 'model_psycho_ring'
print (exp_info)


downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = './tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

In [ ]:
from psychopy import visual, core, event
import MotionClouds as MC
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
from PIL import Image

test_length = 150
MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
model = model.eval()


ans_list = []
N_B_thetas = 7
B_thetas = np.pi*np.logspace(-6, -1, N_B_thetas, base=2)
    
std_theta = np.pi/6

for trial in range(test_length):
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2

    varimgmc2 = data_transform(Image.open(MC2))
    #varimgmc2 = data_transform(Image.open('16_clouds_easy/0.0/B0 1.40625.png'))
    varimgmc2 = Variable(varimgmc2)
    varimgmc2 = varimgmc2.unsqueeze(0)
    
    net_ans = model(varimgmc2)
    pred = net_ans.data.max(1)[1] #prediction
    #print(pred)
    #correct = (np.sign(theta) > 0) and (pred[0]>8)
    if np.sign(theta) > 0 and pred[0] > 8 :
        correct = True
    if np.sign(theta) < 0 and pred[0] <= 8:
        correct = True
    else :
        correct = False
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ', pred[0], '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi))

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    if pred[0] >8 :
        ans_list.append([trial, theta, 'left', B_theta])
    if pred[0] <= 8 :
        ans_list.append([trial, theta, 'right', B_theta])


In [ ]:
import pickle
pickle.dump(ans_list, open('../psychophysics_data/Psy_discrim_final_%s.p' % exp_info, 'wb'))

# Trash

In [52]:
varimgmc2 = data_transform(Image.open('16_clouds_easy/2.722713633111154/B0 10.928413651643268.png'))
varimgmc2 = Variable(varimgmc2)
varimgmc2 = varimgmc2.unsqueeze(0)

net_ans = model(varimgmc2)
print(net_ans.data.max(1)[1])


 13
[torch.LongTensor of size 1]

